In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from category_encoders.target_encoder import TargetEncoder
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import mean_squared_error, r2_score
os.chdir('C:/Users/louka/Desktop/Machine Learning course/Project/Machine-learning-project')
print(os.getcwd())

C:\Users\louka\Desktop\Machine Learning course\Project\Machine-learning-project


In [2]:
def process_percentage_column(column):
    return column.str.rstrip('%').astype(float) / 100

In [27]:
#We will get the 2012,2016,2020 datasets
df_2012=pd.read_csv('Data/Features_2012.csv')
df_2016=pd.read_csv('Data/Features_2016.csv')
df_2021=pd.read_csv('Data/Features_2021.csv')

df_target_2012=pd.read_csv('Data/2012 Election Results.csv')
df_target_2016=pd.read_csv('Data/2016 Presidential Election Results by State.csv')
df_target_2020=pd.read_csv('Data/2020 elections made.csv')

df_test_2023=pd.read_csv('Data/Features_2023.csv')

df_target_2012.drop([51],axis=0,inplace=True)
df_target_2012[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']] = df_target_2012[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']].apply(pd.to_numeric, errors='coerce')
df_target_2012[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']]=df_target_2012[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']].div(100) #fixed

#print(df_target_2012) #nice
df_target_2020[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']]=df_target_2020[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']].apply(process_percentage_column)

#Merge feature data
df_train=pd.concat([df_2012,df_2016,df_2021],ignore_index=True)
# print(df_train)

# #Merge target data
df_target=pd.concat([df_target_2012,df_target_2016,df_target_2020],ignore_index=True)
#print(df_target)
#Make indices nice etc
df_train.set_index("NAME",inplace=True) #nice
df_target.set_index("STATE",inplace=True)
df_test_2023.set_index("NAME",inplace=True)
#print(df_train)

#Fixes:
#GRAPI is wrong for anything other than 2012. RIP
df_train.drop(df_train.columns[df_train.columns.str.contains('GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME')], axis=1, inplace=True)
df_test_2023.drop(df_test_2023.columns[df_test_2023.columns.str.contains('GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME')], axis=1, inplace=True)

#df_train

In [33]:
#XGBoost Regression
X=df_train
y=df_target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)
estimators = [
    ("imputation_mean", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ('clf', XGBRegressor(random_state=8))
]
pipe=Pipeline(steps=estimators)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
print(y_pred)
test_r2=r2_score(y_test,y_pred)
print(test_r2)
df_results=pd.DataFrame(y_pred)
df_results.to_csv(f'Data\Results_2023.csv', index=True)

[[ 5.8764237e-01  4.2208457e-01 -4.4479311e-04]
 [ 6.6405147e-01  2.9864162e-01  5.3207636e-02]
 [ 5.1026726e-01  4.2447793e-01  2.4289617e-02]
 [ 4.2979029e-01  4.6161908e-01  7.7213976e-05]
 [ 6.1258322e-01  4.4474581e-01  2.0710150e-02]
 [ 3.7948176e-01  5.1210403e-01  3.3809431e-02]
 [ 3.4079510e-01  5.5974567e-01  3.3519816e-02]
 [ 4.9558833e-01  4.3185139e-01  4.2652991e-02]
 [ 4.7316393e-01  4.5223954e-01  2.4412265e-02]
 [ 6.0768116e-01  3.9890397e-01  2.3214661e-02]
 [ 5.0349164e-01  4.5410722e-01  1.3812804e-03]
 [ 6.0486197e-01  3.8291955e-01  1.4580834e-02]
 [ 4.6138668e-01  4.9556231e-01  3.5527755e-02]
 [ 4.1797444e-01  5.9514713e-01  4.1693050e-02]
 [ 5.5022734e-01  4.6613246e-01  8.4952917e-05]
 [ 4.7160029e-01  4.4925895e-01  2.6956921e-02]
 [ 3.5453191e-01  5.1257610e-01  4.1193441e-02]
 [ 5.4365855e-01  4.2815158e-01  1.4883623e-02]
 [ 5.6567830e-01  3.8201475e-01  3.3742640e-02]
 [ 4.7854140e-01  3.6013213e-01  4.0924989e-02]
 [ 3.7357980e-01  5.6302679e-01 -4.56653

PermissionError: [Errno 13] Permission denied: 'Data\\Results_2023.csv'

In [5]:
search_space = {
    'clf__max_depth': Integer(2,8),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.5, 1.0),
    'clf__colsample_bytree': Real(0.5, 1.0),
    'clf__colsample_bylevel': Real(0.5, 1.0),
    'clf__colsample_bynode' : Real(0.5, 1.0),
    'clf__reg_alpha': Real(0.0, 10.0),
    'clf__reg_lambda': Real(0.0, 10.0),
    'clf__gamma': Real(0.0, 10.0)
}
                                       #cv: number of folds
opt = BayesSearchCV(pipe, search_space, cv=3, n_iter=5, scoring='r2', random_state=8) 
# in reality, you may consider setting cv and n_iter to higher values

In [6]:
opt.fit(X_train, y_train)
best_model = opt.best_estimator_
y_pred = best_model.predict(X_test)
test_r2 = r2_score(y_test, y_pred)
# Output results

print("Best hyperparameters:", opt.best_params_)
print("R^2 score on the test set:", test_r2)

Best hyperparameters: OrderedDict([('clf__colsample_bylevel', 0.7750018497221565), ('clf__colsample_bynode', 0.5614437441596264), ('clf__colsample_bytree', 0.9126202065825759), ('clf__gamma', 8.289497472648083), ('clf__learning_rate', 0.4299244814327041), ('clf__max_depth', 6), ('clf__reg_alpha', 2.784887532399771), ('clf__reg_lambda', 1.67027558902639), ('clf__subsample', 0.5966102807384807)])
R^2 score on the test set: -7.2189106941223145
